In [ ]:
import scipy.stats
import sklearn.model_selection
import tensorflow as tf
import itertools
import numpy as np
import scipy.interpolate
import glob
import re
import os
import pandas as pd
import os
import pickle
import scipy
import datetime
import numpy as np
# Libraries for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
import sklearn.manifold
import requests
import zipfile
#importing necessary libraries
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import scipy
from scipy.stats import entropy

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn import preprocessing 

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder,normalize, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,recall_score  
from sklearn.manifold import TSNE

import plotly.express as px
import plotly.graph_objects as go
import optuna
from sklearn.metrics import confusion_matrix
import seaborn as sns
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset
from sklearn.manifold import TSNE

# import some libraries
import pandas as pd
from sklearn.preprocessing import Normalizer,MinMaxScaler
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import random
import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder,normalize, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import shutil, time, os, requests, random, copy
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from collections import Counter
import joblib
%matplotlib inline

In [ ]:
!pip install fvcore

In [ ]:
def set_seed(seed_value):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    sklearn.random.seed(seed_value)
    sklearn.utils.check_random_state(seed_value)
    joblib.parallel_backend('threading', n_jobs=1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# Set seed for reproducibility
SEED = 42
set_seed(SEED)

# **Download and extract zip file**

In [ ]:
dataset_url = 'https://github.com/mmalekzadeh/motion-sense/blob/master/data/B_Accelerometer_data.zip?raw=true'
working_directory = "/kaggle/working/"
r = requests.get(dataset_url, allow_redirects=True)
with open(working_directory + 'B_Accelerometer_data.zip', 'wb') as f:
    f.write(r.content)

In [ ]:
with zipfile.ZipFile(working_directory + 'B_Accelerometer_data.zip', 'r') as zip_ref:
    zip_ref.extractall(working_directory)

# **Raw Data Processing func**

In [ ]:
def process_motion_sense_accelerometer_files(accelerometer_data_folder_path):
    """
    Preprocess the accelerometer files of the MotionSense dataset into the 'user-list' format
    Data files can be found at https://github.com/mmalekzadeh/motion-sense/tree/master/data

    Parameters:

        accelerometer_data_folder_path (str):
            the path to the folder containing the data files (unzipped)
            e.g. motionSense/B_Accelerometer_data/
            the trial folders should be directly inside it (e.g. motionSense/B_Accelerometer_data/dws_1/)

    Return:
        
        user_datsets (dict of {user_id: [(sensor_values, activity_labels)]})
            the processed dataset in a dictionary, of type {user_id: [(sensor_values, activity_labels)]}
            the keys of the dictionary is the user_id (participant id)
            the values of the dictionary are lists of (sensor_values, activity_labels) pairs
                sensor_values are 2D numpy array of shape (length, channels=3)
                activity_labels are 1D numpy array of shape (length)
                each pair corresponds to a separate trial 
                    (i.e. time is not contiguous between pairs, which is useful for making sliding windows, where it is easy to separate trials)
    """

    # label_set = {}
    user_datasets = {}
    all_trials_folders = sorted(glob.glob(accelerometer_data_folder_path + "/*"))

    # Loop through every trial folder
    for trial_folder in all_trials_folders:
        trial_name = os.path.split(trial_folder)[-1]

        # label of the trial is given in the folder name, separated by underscore
        label = trial_name.split("_")[0]
        # label_set[label] = True
        print(trial_folder)
        
        # Loop through files for every user of the trail
        for trial_user_file in sorted(glob.glob(trial_folder + "/*.csv")):

            # use regex to match the user id
            user_id_match = re.search(r'(?P<user_id>[0-9]+)\.csv', os.path.split(trial_user_file)[-1])
            if user_id_match is not None:
                user_id = int(user_id_match.group('user_id'))

                # Read file
                user_trial_dataset = pd.read_csv(trial_user_file)
                user_trial_dataset.dropna(how = "any", inplace = True)

                # Extract the x, y, z channels
                values = user_trial_dataset[["x", "y", "z"]].values

                # the label is the same during the entire trial, so it is repeated here to pad to the same length as the values
                labels = np.repeat(label, values.shape[0])

                if user_id not in user_datasets:
                    user_datasets[user_id] = []
                user_datasets[user_id].append((values, labels))
            else:
                print("[ERR] User id not found", trial_user_file)
    
    return user_datasets

# **Data Pre-Processing func**

In [ ]:
def get_mode(np_array):
    
    """
    Get the mode (majority/most frequent value) from a 1D array
    """
    frequency_dict = Counter(np_array)
    max_key = max(frequency_dict, key=frequency_dict.get)
#     return scipy.stats.mode(np_array)[0]
    return max_key


def sliding_window_np(X, window_size, shift, stride, offset=0, flatten=None):
    """
    Create sliding windows from an ndarray

    Parameters:
    
        X (numpy-array)
            The numpy array to be windowed
        
        shift (int)
            number of timestamps to shift for each window
            (200 here refers to 50% overlap, no overlap if =400)

        stride (int)
            stride of the window (dilation)

        offset (int)
            starting index of the first window
        
        flatten (function (array) -> (value or array) )
            the function to be applied to a window after it is extracted
            can be used with get_mode (see above) for extracting the label by majority voting
            ignored if is None

    Return:

        Windowed ndarray
            shape[0] is the number of windows
    """

    overall_window_size = (window_size - 1) * stride + 1
    num_windows = (X.shape[0] - offset - (overall_window_size)) // shift + 1
    windows = []
    for i in range(num_windows):
        start_index = i * shift + offset
        this_window = X[start_index : start_index + overall_window_size : stride]
        if flatten is not None:
            this_window = flatten(this_window)
        windows.append(this_window)
    return np.array(windows)


def get_windows_dataset_from_user_list_format(user_datasets, window_size=200, shift=200, stride=1, verbose=0):
    """
    Create windows dataset in 'user-list' format using sliding windows

    Parameters:

        user_datasets
            dataset in the 'user-list' format {user_id: [(sensor_values, activity_labels)]}
        
        window_size = 400
            size of the window (output)

        shift = 200
            number of timestamps to shift for each window
            (200 here refers to 50% overlap, no overlap if =400)

        stride = 1
            stride of the window (dilation)

        verbose = 0
            debug messages are printed if > 0

    
    Return:

        user_dataset_windowed
            Windowed version of the user_datasets
            Windows from different trials are combined into one array
            type: {user_id: ( windowed_sensor_values, windowed_activity_labels)}
            windowed_sensor_values have shape (num_window, window_size, channels)
            windowed_activity_labels have shape (num_window)

            Labels are decided by majority vote
    """
    
    user_dataset_windowed = {}

    for user_id in user_datasets:
        if verbose > 0:
            print(f"Processing {user_id}")
        x = []
        y = []

        # Loop through each trail of each user
        for v,l in user_datasets[user_id]:
            v_windowed = sliding_window_np(v, window_size, shift, stride)
            
            # flatten the window by majority vote (1 value for each window)
            l_flattened = sliding_window_np(l, window_size, shift, stride, flatten=get_mode)
            if len(v_windowed) > 0:
                x.append(v_windowed)
                y.append(l_flattened)
            if verbose > 0:
                print(f"Data: {v_windowed.shape}, Labels: {l_flattened.shape}")

        # combine all trials
        user_dataset_windowed[user_id] = (np.concatenate(x), np.concatenate(y).squeeze())
    return user_dataset_windowed


def combine_windowed_dataset(user_datasets_windowed, train_users, test_users=None, verbose=0):
    """
    Combine a windowed 'user-list' dataset into training and test sets

    Parameters:

        user_dataset_windowed
            dataset in the windowed 'user-list' format {user_id: ( windowed_sensor_values, windowed_activity_labels)}
        
        train_users
            list or set of users (corresponding to the user_id) to be used as training data

        test_users = None
            list or set of users (corresponding to the user_id) to be used as testing data
            if is None, then all users not in train_users will be treated as test users 

        verbose = 0
            debug messages are printed if > 0

    Return:
        (train_x, train_y, test_x, test_y)
            train_x, train_y
                the resulting training/test input values as a single numpy array
            test_x, test_y
                the resulting training/test labels as a single (1D) numpy array
    """
    
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    for user_id in user_datasets_windowed:
        
        v,l = user_datasets_windowed[user_id]
        if user_id in train_users:
            if verbose > 0:
                print(f"{user_id} Train")
            train_x.append(v)
            train_y.append(l)
        elif test_users is None or user_id in test_users:
            if verbose > 0:
                print(f"{user_id} Test")
            test_x.append(v)
            test_y.append(l)
    

    if len(train_x) == 0:
        train_x = np.array([])
        train_y = np.array([])
    else:
        train_x = np.concatenate(train_x)
        train_y = np.concatenate(train_y).squeeze()
    
    if len(test_x) == 0:
        test_x = np.array([])
        test_y = np.array([])
    else:
        test_x = np.concatenate(test_x)
        test_y = np.concatenate(test_y).squeeze()

    return train_x, train_y, test_x, test_y


def get_mean_std_from_user_list_format(user_datasets, train_users):
    """
    Obtain and means and standard deviations from a 'user-list' dataset (channel-wise)
    from training users only

    Parameters:

        user_datasets
            dataset in the 'user-list' format {user_id: [(sensor_values, activity_labels)]}
        
        train_users
            list or set of users (corresponding to the user_ids) from which the mean and std are extracted

    Return:
        (means, stds)
            means and stds of the particular users (channel-wise)
            shape: (num_channels)

    """
    
    mean_std_data = []
    for u in train_users:
        for data, _ in user_datasets[u]:
            mean_std_data.append(data)
    mean_std_data_combined = np.concatenate(mean_std_data)
    means = np.mean(mean_std_data_combined, axis=0)
    stds = np.std(mean_std_data_combined, axis=0)
    return (means, stds)


def normalise(data, mean, std):
    """
    Normalise data (Z-normalisation)
    """

    return ((data - mean) / std)


def apply_label_map(y, label_map):
    """
    Apply a dictionary mapping to an array of labels
    Can be used to convert str labels to int labels

    Parameters:
        y
            1D array of labels
        label_map
            a label dictionary of (label_original -> label_new)

    Return:
        y_mapped
            1D array of mapped labels
            None values are present if there is no entry in the dictionary
    """

    y_mapped = []
    for l in y:
        y_mapped.append(label_map.get(l))
    return np.array(y_mapped)


def filter_none_label(X, y):
    """
    Filter samples of the value None
    Can be used to exclude non-mapped values from apply_label_map

    Parameters:
        X
            data values

        y
            labels (1D)

    Return:
        (X_filtered, y_filtered)
            X_filtered
                filtered data values
            
            y_filtered
                filtered labels (of type int)
    """

    valid_mask = np.where(y != None)
    return (np.array(X[valid_mask]), np.array(y[valid_mask], dtype=int))


def pre_process_dataset_composite(user_datasets, label_map, output_shape, train_users, test_users, window_size, shift, normalise_dataset=True, validation_split_proportion=0.2, verbose=0):
    """
    A composite function to process a dataset
    Steps
        1: Use sliding window to make a windowed dataset (see get_windows_dataset_from_user_list_format)
        2: Split the dataset into training and test set (see combine_windowed_dataset)
        3: Normalise the datasets (see get_mean_std_from_user_list_format)
        4: Apply the label map and filter labels (see apply_label_map, filter_none_label)
        5: One-hot encode the labels (see tf.keras.utils.to_categorical)
        6: Split the training set into training and validation sets (see sklearn.model_selection.train_test_split)
    
    Parameters:
        user_datasets
            dataset in the 'user-list' format {user_id: [(sensor_values, activity_labels)]}

        label_map
            a mapping of the labels
            can be used to filter labels
            (see apply_label_map and filter_none_label)

        output_shape
            number of output classifiction categories
            used in one hot encoding of the labels
            (see tf.keras.utils.to_categorical)

        train_users
            list or set of users (corresponding to the user_id) to be used as training data

        test_users
            list or set of users (corresponding to the user_id) to be used as testing data

        window_size
            size of the data windows
            (see get_windows_dataset_from_user_list_format)

        shift
            number of timestamps to shift for each window
            (see get_windows_dataset_from_user_list_format)

        normalise_dataset = True
            applies Z-normalisation if True

        validation_split_proportion = 0.2
            if not None, the proportion for splitting the full training set further into training and validation set using random sampling
            (see sklearn.model_selection.train_test_split)
            if is None, the training set will not be split - the return value np_val will also be none

        verbose = 0
            debug messages are printed if > 0

    
    Return:
        (np_train, np_val, np_test)
            three pairs of (X, y)
            X is a windowed set of data points
            y is an array of one-hot encoded labels

            if validation_split_proportion is None, np_val is None
    """

    # Step 1
    user_datasets_windowed = get_windows_dataset_from_user_list_format(user_datasets, window_size=window_size, shift=shift)

    # Step 2
    train_x, train_y, test_x, test_y = combine_windowed_dataset(user_datasets_windowed, train_users)

    # Step 3
    if normalise_dataset:
        means, stds = get_mean_std_from_user_list_format(user_datasets, train_users)
        train_x = normalise(train_x, means, stds)
        test_x = normalise(test_x, means, stds)

    # Step 4
    train_y_mapped = apply_label_map(train_y, label_map)
    test_y_mapped = apply_label_map(test_y, label_map)

    train_x, train_y_mapped = filter_none_label(train_x, train_y_mapped)
    test_x, test_y_mapped = filter_none_label(test_x, test_y_mapped)

    if verbose > 0:
        print("Test")
        print(np.unique(test_y, return_counts=True))
        print(np.unique(test_y_mapped, return_counts=True))
        print("-----------------")

        print("Train")
        print(np.unique(train_y, return_counts=True))
        print(np.unique(train_y_mapped, return_counts=True))
        print("-----------------")

    # Step 5
    train_y_one_hot = tf.keras.utils.to_categorical(train_y_mapped, num_classes=output_shape)
    test_y_one_hot = tf.keras.utils.to_categorical(test_y_mapped, num_classes=output_shape)

    r = np.random.randint(len(train_y_mapped))
    assert train_y_one_hot[r].argmax() == train_y_mapped[r]
    r = np.random.randint(len(test_y_mapped))
    assert test_y_one_hot[r].argmax() == test_y_mapped[r]

    # Step 6
    if validation_split_proportion is not None and validation_split_proportion > 0:
        train_x_split, val_x_split, train_y_split, val_y_split = sklearn.model_selection.train_test_split(train_x, train_y_one_hot, test_size=validation_split_proportion, random_state=42)
    else:
        train_x_split = train_x
        train_y_split = train_y_one_hot
        val_x_split = None
        val_y_split = None
        

    if verbose > 0:
        print("Training data shape:", train_x_split.shape)
        print("Validation data shape:", val_x_split.shape if val_x_split is not None else "None")
        print("Testing data shape:", test_x.shape)

    np_train = (train_x_split, train_y_split)
    np_val = (val_x_split, val_y_split) if val_x_split is not None else None
    np_test = (test_x, test_y_one_hot)

    # original_np_train = np_train
    # original_np_val = np_val
    # original_np_test = np_test

    return (np_train, np_val, np_test)


def pre_process_dataset_composite_in_user_format(user_datasets, label_map, output_shape, train_users, window_size, shift, normalise_dataset=True, verbose=0):
    """
    A composite function to process a dataset which outputs processed datasets separately for each user (of type: {user_id: ( windowed_sensor_values, windowed_activity_labels)}).
    This is different from pre_process_dataset_composite where the data from the training and testing users are not combined into one object.

    Steps
        1: Use sliding window to make a windowed dataset (see get_windows_dataset_from_user_list_format)
        For each user:
            2: Apply the label map and filter labels (see apply_label_map, filter_none_label)
            3: One-hot encode the labels (see tf.keras.utils.to_categorical)
            4: Normalise the data (see get_mean_std_from_user_list_format)
    
    Parameters:
        user_datasets
            dataset in the 'user-list' format {user_id: [(sensor_values, activity_labels)]}

        label_map
            a mapping of the labels
            can be used to filter labels
            (see apply_label_map and filter_none_label)

        output_shape
            number of output classifiction categories
            used in one hot encoding of the labels
            (see tf.keras.utils.to_categorical)

        train_users
            list or set of users (corresponding to the user_id) to be used for normalising the dataset

        window_size
            size of the data windows
            (see get_windows_dataset_from_user_list_format)

        shift
            number of timestamps to shift for each window
            (see get_windows_dataset_from_user_list_format)

        normalise_dataset = True
            applies Z-normalisation if True

        verbose = 0
            debug messages are printed if > 0

    
    Return:
        user_datasets_processed
            Processed version of the user_datasets in the windowed format
            type: {user_id: (windowed_sensor_values, windowed_activity_labels)}
    """

    # Preparation for step 2
    if normalise_dataset:
        means, stds = get_mean_std_from_user_list_format(user_datasets, train_users)

    # Step 1
    user_datasets_windowed = get_windows_dataset_from_user_list_format(user_datasets, window_size=window_size, shift=shift)

    
    user_datasets_processed = {}
    for user, user_dataset in user_datasets_windowed.items():
        data, labels = user_dataset

        # Step 2
        labels_mapped = apply_label_map(labels, label_map)
        data_filtered, labels_filtered = filter_none_label(data, labels_mapped)

        # Step 3
        labels_one_hot = tf.keras.utils.to_categorical(labels_filtered, num_classes=output_shape)

        # random check
        r = np.random.randint(len(labels_filtered))
        assert labels_one_hot[r].argmax() == labels_filtered[r]

        # Step 4
        if normalise_dataset:
            data_filtered = normalise(data_filtered, means, stds)

        user_datasets_processed[user] = (data_filtered, labels_one_hot)

        if verbose > 0:
            print("Data shape of user", user, ":", data_filtered.shape)
    
    return user_datasets_processed


def add_user_id_to_windowed_dataset(user_datasets_windowed, encode_user_id=True, as_feature=False, as_label=True, verbose=0):
    """
    Add user ids as features or labels to a windowed dataset
    The user ids are appended to the last dimension of the arrays
    E.g. sensor values of shape (100, 400, 3) will become (100, 400, 4), and data[:, :, -1] will contain the user id
    Similarly labels of shape (100, 5) will become (100, 6), and labels[:, -1] will contain the user id
    
    Parameters:
        user_datasets_windowed
            dataset in the 'windowed-user' format type: {user_id: (windowed_sensor_values, windowed_activity_labels)}

        encode_user_id = True
            whether to encode the user ids as integers
            if True: 
                encode all user ids as integers when being appended to the np arrays
                return the map from user id to integer as an output
                note that the dtype of the output np arrays will be kept as float if they are originally of type float
            if False:
                user ids will be kept as is when being appended to the np arrays
                WARNING: if the user id is of type string, the output arrays will also be converted to type string, which might be difficult to work with

        as_feature = False
            user ids will be added to the windowed_sensor_values arrays as extra features if True

        as_label = False
            user ids will be added to the windowed_activity_labels arrays as extra labels if True

        verbose = 0
            debug messages are printed if > 0

    Return:
        user_datasets_modified, user_id_encoder

            user_datasets_modified
                the modified version of the input (user_datasets_windowed)
                with the same type {user_id: ( windowed_sensor_values, windowed_activity_labels)}
            user_id_encoder
                the encoder which maps user ids to integers
                type: {user_id: encoded_user_id}
                None if encode_user_id is False
    """

    # Create the mapping from user_id to integers
    if encode_user_id:
        all_users = sorted(list(user_datasets_windowed.keys()))
        user_id_encoder = dict([(u, i) for i, u in enumerate(all_users)])
    else:
        user_id_encoder = None

    # if none of the options are enabled, return the input
    if not as_feature and not as_label:
        return user_datasets_windowed, user_id_encoder

    user_datasets_modified = {}
    for user, user_dataset in user_datasets_windowed.items():
        data, labels = user_dataset

        # Get the encoded user_id
        if encode_user_id:
            user_id = user_id_encoder[user]
        else:
            user_id = user

        # Add user_id as an extra feature
        if as_feature:
            user_feature = np.expand_dims(np.full(data.shape[:-1], user_id), axis=-1)
            data_modified = np.append(data, user_feature, axis=-1)
        else:
            data_modified = data
        
        # Add user_id as an extra label
        if as_label:
            user_labels = np.expand_dims(np.full(labels.shape[:-1], user_id), axis=-1)
            labels_modified = np.append(labels, user_labels, axis=-1)
        else:
            labels_modified = labels

        if verbose > 0:
            print(f"User {user}: id {repr(user)} -> {repr(user_id)}, data shape {data.shape} -> {data_modified.shape}, labels shape {labels.shape} -> {labels_modified.shape}")

        user_datasets_modified[user] = (data_modified, labels_modified)
    
    return user_datasets_modified, user_id_encoder


def make_batches_reshape(data, batch_size):
    """
    Make a batched dataset from a windowed time-series by simple reshaping
    Note that the last batch is dropped if incomplete

    Parameters:
        data
            A 3D numpy array in the shape (num_windows, window_size, num_channels)

        batch_size
            the (maximum) size of the batches

    Returns:
        batched_data
            A 4D numpy array in the shape (num_batches, batch_size, window_size, num_channels)
    """

    max_len = (data.shape[0]) // batch_size * batch_size
    return data[:max_len].reshape((-1, batch_size, data.shape[-2], data.shape[-1]))


def np_random_shuffle_index(length):
    """
    Get a list of randomly shuffled indices
    """
    indices = np.arange(length)
    np.random.shuffle(indices)
    return indices


def ceiling_division(n, d):
    """
    Ceiling integer division
    """
    return -(n // -d)


def get_batched_dataset_generator(data, batch_size):
    """
    Create a data batch generator
    Note that the last batch might not be full

    Parameters:
        data
            A numpy array of data

        batch_size
            the (maximum) size of the batches

    Returns:
        generator<numpy array>
            a batch of the data with the same shape except the first dimension, which is now the batch size
    """

    num_bathes = ceiling_division(data.shape[0], batch_size)
    for i in range(num_bathes):
        yield data[i * batch_size : (i + 1) * batch_size]

    # return data[:max_len].reshape((-1, batch_size, data.shape[-2], data.shape[-1]))

In [ ]:
dataset_save_path = working_directory
if not os.path.exists(working_directory):
    os.mkdir(working_directory)

In [ ]:
accelerometer_data_folder_path = working_directory + 'B_Accelerometer_data'
# accelerometer_data_folder_path = "/kaggle/working/B_Accelerometer_data"
user_datasets = process_motion_sense_accelerometer_files(accelerometer_data_folder_path)

In [ ]:
with open(working_directory + 'motion_sense_user_split.pkl', 'wb') as f:
    pickle.dump({
        'user_split': user_datasets,
    }, f)

# **pre processing**

In [ ]:
# Parameters
window_size = 500
input_shape = (window_size, 3)

# Dataset Metadata 
transformation_multiple = 1
dataset_name = 'motion_sense.pkl'
dataset_name_user_split = 'motion_sense_user_split.pkl'

label_list = ['null', 'sit', 'std', 'wlk', 'ups', 'dws', 'jog']
label_list_full_name = ['null', 'sitting', 'standing', 'walking', 'walking upstairs', 'walking downstairs', 'jogging']
has_null_class = True

label_map = dict([(l, i) for i, l in enumerate(label_list)])

output_shape = len(label_list)

model_save_name = f"motionsense_acc"

sampling_rate = 50.0
unit_conversion = scipy.constants.g

# a fixed user-split
test_users_fixed = [1, 14, 19, 23, 6]
def get_fixed_split_users(har_users):
#     print(har_users)
#     test_users = har_users[0::5]
    test_users = test_users_fixed
    train_users = [u for u in har_users if u not in test_users]
#     train_users = [10, 11, 12, 13, 15, 16, 17, 18, 2, 20, 21, 22, 24, 3, 4, 5, 7, 8]
#     test_users = [1, 14, 19, 23, 6, 9]
    return (train_users, test_users)

In [ ]:
dataset_name_user_split

In [ ]:
dataset_save_path = "/kaggle/working/"

In [ ]:
with open(dataset_save_path + dataset_name_user_split, 'rb') as f:
    dataset_dict = pickle.load(f)
    user_datasets = dataset_dict['user_split']

In [ ]:
har_users = list(user_datasets.keys())
train_users, test_users = get_fixed_split_users(har_users)
print(f'Testing: {test_users}, Training: {train_users}')

In [ ]:
np_train, np_val, np_test = pre_process_dataset_composite(
    user_datasets = user_datasets,
    label_map = label_map,
    output_shape = output_shape,
    train_users = train_users,
    test_users = test_users,
    window_size = window_size,
    shift = window_size // 2,
    normalise_dataset = True,
    verbose = 1
)

In [ ]:
train_data = np.vstack((np_train[0], np_val[0]))
train_label = np.vstack((np_train[1], np_val[1]))

In [ ]:
train_data.shape

In [ ]:
train_label.shape

In [ ]:
np_train=(train_data, train_label)

In [ ]:
np_train[0].shape

In [ ]:
import numpy as np
from scipy.stats import entropy

def calculate_rms(data):
    """Compute Root Mean Squared (RMS)"""
    return np.sqrt(np.mean(data**2, axis=1))

def calculate_p2p(data):
    """Compute Peak-to-Peak"""
    return np.abs(np.max(data, axis=1)) + np.abs(np.min(data, axis=1))

def calculate_entropy(data, bins=500):
    """Compute Shannon Entropy after binning"""
    ent = np.zeros((data.shape[0], data.shape[2]))  # (N, 3)
    for i in range(data.shape[2]):  # Iterate over channels
        counts = np.apply_along_axis(lambda x: np.histogram(x, bins=bins)[0], 1, data[:, :, i])
        ent[:, i] = entropy(counts, axis=1)
    return ent

def calculate_clearance(data):
    """Compute Clearance Factor"""
    return (np.mean(np.sqrt(np.abs(data)), axis=1))**2

def extract_time_features(data):
    """Extracts 12 features per channel (3 channels) -> Output shape (N, 36)"""
    N, _, C = data.shape  # (N, 200, 3)
    
    mean_abs = np.mean(np.abs(data), axis=1)
    std_dev = np.std(data, axis=1)
    skewness = np.mean(((data - np.mean(data, axis=1, keepdims=True)) / std_dev[:, np.newaxis, :])**3, axis=1)
    kurt = np.mean(((data - np.mean(data, axis=1, keepdims=True)) / std_dev[:, np.newaxis, :])**4, axis=1) - 3
    rms = calculate_rms(data)
    max_abs = np.max(np.abs(data), axis=1)
    p2p = calculate_p2p(data)
    crest = max_abs / rms
    clearance = calculate_clearance(data)
    shape = rms / mean_abs
    impulse = max_abs / mean_abs
    ent = calculate_entropy(data)

    # Stack all features along the last dimension
    features = np.hstack([mean_abs, std_dev, skewness, kurt, ent, rms, max_abs, p2p, crest, clearance, shape, impulse])  # (N, 36)
    
    return features


In [ ]:
features_train = extract_time_features(np_train[0])
features_test = extract_time_features(np_test[0])


In [ ]:
np.argmax(np_train[1], axis=1).shape

In [ ]:
labels_train = np.argmax(np_train[1], axis=1)
labels_test  = np.argmax(np_test[1], axis=1)

In [ ]:
labels_train = labels_train - 1
labels_test = labels_test - 1
Counter(labels_train)

In [ ]:
features_test.shape

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train)

x_train_standardized = scaler.transform(features_train)
x_test_standardized = scaler.transform(features_test)

print(np.mean(x_train_standardized), np.mean(x_test_standardized))
print(np.std(x_train_standardized), np.std(x_test_standardized))

In [ ]:
features_train.shape

In [ ]:
class KANLinear(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1, self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order : -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                # torch.nn.init.constant_(self.spline_scaler, self.scale_spline)
                torch.nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        """
        Compute the B-spline bases for the given input tensor.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).

        Returns:
            torch.Tensor: B-spline bases tensor of shape (batch_size, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features

        grid: torch.Tensor = (
            self.grid
        )  # (in_features, grid_size + 2 * spline_order + 1)
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:, : -(k + 1)])
                / (grid[:, k:-1] - grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (grid[:, k + 1 :] - x)
                / (grid[:, k + 1 :] - grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )

        assert bases.size() == (
            x.size(0),
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        """
        Compute the coefficients of the curve that interpolates the given points.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
            y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

        Returns:
            torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        assert y.size() == (x.size(0), self.in_features, self.out_features)

        A = self.b_splines(x).transpose(
            0, 1
        )  # (in_features, batch_size, grid_size + spline_order)
        B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
        solution = torch.linalg.lstsq(
            A, B
        ).solution  # (in_features, grid_size + spline_order, out_features)
        result = solution.permute(
            2, 0, 1
        )  # (out_features, in_features, grid_size + spline_order)

        assert result.size() == (
            self.out_features,
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        assert x.size(-1) == self.in_features
        original_shape = x.shape
        x = x.view(-1, self.in_features)

        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        output = base_output + spline_output
        
        output = output.view(*original_shape[:-1], self.out_features)
        return output

    @torch.no_grad()
    def update_grid(self, x: torch.Tensor, margin=0.01):
        assert x.dim() == 2 and x.size(1) == self.in_features
        batch = x.size(0)

        splines = self.b_splines(x)  # (batch, in, coeff)
        splines = splines.permute(1, 0, 2)  # (in, batch, coeff)
        orig_coeff = self.scaled_spline_weight  # (out, in, coeff)
        orig_coeff = orig_coeff.permute(1, 2, 0)  # (in, coeff, out)
        unreduced_spline_output = torch.bmm(splines, orig_coeff)  # (in, batch, out)
        unreduced_spline_output = unreduced_spline_output.permute(
            1, 0, 2
        )  # (batch, in, out)

        # sort each channel individually to collect data distribution
        x_sorted = torch.sort(x, dim=0)[0]
        grid_adaptive = x_sorted[
            torch.linspace(
                0, batch - 1, self.grid_size + 1, dtype=torch.int64, device=x.device
            )
        ]

        uniform_step = (x_sorted[-1] - x_sorted[0] + 2 * margin) / self.grid_size
        grid_uniform = (
            torch.arange(
                self.grid_size + 1, dtype=torch.float32, device=x.device
            ).unsqueeze(1)
            * uniform_step
            + x_sorted[0]
            - margin
        )

        grid = self.grid_eps * grid_uniform + (1 - self.grid_eps) * grid_adaptive
        grid = torch.concatenate(
            [
                grid[:1]
                - uniform_step
                * torch.arange(self.spline_order, 0, -1, device=x.device).unsqueeze(1),
                grid,
                grid[-1:]
                + uniform_step
                * torch.arange(1, self.spline_order + 1, device=x.device).unsqueeze(1),
            ],
            dim=0,
        )

        self.grid.copy_(grid.T)
        self.spline_weight.data.copy_(self.curve2coeff(x, unreduced_spline_output))

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        """
        Compute the regularization loss.

        This is a dumb simulation of the original L1 regularization as stated in the
        paper, since the original one requires computing absolutes and entropy from the
        expanded (batch, in_features, out_features) intermediate tensor, which is hidden
        behind the F.linear function if we want an memory efficient implementation.

        The L1 regularization is now computed as mean absolute value of the spline
        weights. The authors implementation also includes this term in addition to the
        sample-based regularization.
        """
        l1_fake = self.spline_weight.abs().mean(-1)
        regularization_loss_activation = l1_fake.sum()
        p = l1_fake / regularization_loss_activation
        regularization_loss_entropy = -torch.sum(p * p.log())
        return (
            regularize_activation * regularization_loss_activation
            + regularize_entropy * regularization_loss_entropy
        )


class KAN(torch.nn.Module):
    def __init__(
        self,
        layers_hidden,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KAN, self).__init__()
        self.grid_size = grid_size
        self.spline_order = spline_order

        self.layers = torch.nn.ModuleList()
        for in_features, out_features in zip(layers_hidden, layers_hidden[1:]):
            self.layers.append(
                KANLinear(
                    in_features,
                    out_features,
                    grid_size=grid_size,
                    spline_order=spline_order,
                    scale_noise=scale_noise,
                    scale_base=scale_base,
                    scale_spline=scale_spline,
                    base_activation=base_activation,
                    grid_eps=grid_eps,
                    grid_range=grid_range,
                )
            )

    def forward(self, x: torch.Tensor, update_grid=False):
        for layer in self.layers:
            if update_grid:
                layer.update_grid(x)
            x = layer(x)
        return x

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        return sum(
            layer.regularization_loss(regularize_activation, regularize_entropy)
            for layer in self.layers
        )

In [ ]:
def Create_Dataloader_final1(x_train, x_test, y_train, y_test, batch_size = 8):
    
    X_train, X_valid, y_train, y_valid = x_train, x_test, y_train, y_test

    trainX = np.array(X_train)
    y_train = np.array(y_train)
    validX = np.array(X_valid)
    y_valid = np.array(y_valid)
    
    
    # reshaping data
    X_train1 = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
    X_valid1 = np.reshape(validX, (validX.shape[0],validX.shape[1],1))

    # Create the DataLoader for our data
    X_train1 = torch.tensor(X_train1)
    X_valid1 = torch.tensor(X_valid1)
    
    y_train = torch.tensor(y_train).type(torch.LongTensor)
    y_valid = torch.tensor(y_valid).type(torch.LongTensor)

    train_data = TensorDataset(X_train1, y_train)
    valid_data = TensorDataset(X_valid1, y_valid)

    train_sampler = RandomSampler(train_data)
    valid_sampler = RandomSampler(valid_data)

    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, drop_last=True)
    valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size, drop_last=True)
    return train_dataloader, valid_dataloader


In [ ]:
def final_evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    val_ep_loss = []
    val_ep_acc = []
    val_ep_f1=[]
    val_ep_precision=[]
    val_ep_recall=[]

    model.eval()
    loss_sublist = np.array([])
    acc_sublist = np.array([])
    f1_sublist = np.array([])
    precision_sublist = np.array([])
    recall_sublist = np.array([])

    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        for x,y in val_dataloader:

            z = model(x.to(device).float())
            val_loss = loss_fn(z,y.to(device).squeeze())

            # model's prediction
            preds = torch.exp(z.cpu().data)/torch.sum(torch.exp(z.cpu().data))
            loss_sublist = np.append(loss_sublist, val_loss.cpu().data)
            acc_sublist = np.append(acc_sublist,np.array(np.argmax(preds,axis=1)==y.cpu().data.view(-1)).astype('int'),axis=0)
            f1_sublist = np.append(f1_sublist,f1_score(np.array(y.data.view(-1).reshape(-1,1)),np.array((np.argmax(preds,axis=1)).reshape(-1,1)),average="macro",zero_division=0))
            precision_sublist = np.append(precision_sublist,precision_score(np.array(y.data.view(-1).reshape(-1,1)),np.array((np.argmax(preds,axis=1)).reshape(-1,1)),average="macro",zero_division=0))
            recall_sublist = np.append(recall_sublist,recall_score(np.array(y.data.view(-1).reshape(-1,1)),np.array((np.argmax(preds,axis=1)).reshape(-1,1)),average="macro",zero_division=0))

    return np.mean(loss_sublist), np.mean(acc_sublist), np.mean(f1_sublist), np.mean(precision_sublist), np.mean(recall_sublist)

In [ ]:
def final_train(model, optimizer, ep, train_dataloader_final, val_dataloader_final, evaluation=False):
    """Train the final model.
    """

    # Start training loop
    print("Start training...\n")
    tr_ep_loss = []
    tr_ep_acc = []
    tr_ep_f1 = []
    tr_ep_precision = []
    tr_ep_recall = []

    val_ep_loss = []
    val_ep_acc = []
    val_ep_f1 = []
    val_ep_precision = []
    val_ep_recall = []

    best_val_f1 = 0
    loss_fn = nn.CrossEntropyLoss()

    for e in range(1, ep + 1):
        # =======================================
        #               Training
        # =======================================
        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()
        
        # For each batch of training data...
        loss_sublist = np.array([])
        acc_sublist = np.array([])
        f1_sublist = np.array([])
        precision_sublist = np.array([])
        recall_sublist = np.array([])

        for batch, (x, y) in enumerate(train_dataloader_final):
#             x.to(device)
#             y.to(device)
            batch_counts +=1
            optimizer.zero_grad()

            # positive pair, with encoding
            z = model_final(x.to(device).float())
            #y_true.append(y.squeeze(1).numpy())

            # calculate loss
            loss = loss_fn(z, y.to(device).squeeze())
            loss.backward()

            preds = torch.exp(z.cpu().data)/torch.sum(torch.exp(z.cpu().data))
            loss_sublist = np.append(loss_sublist, loss.cpu().data)
            acc_sublist = np.append(acc_sublist,np.array(np.argmax(preds,axis=1)==y.cpu().data.view(-1)).astype('int'),axis=0)
            f1_sublist = np.append(f1_sublist,f1_score(np.array(y.data.view(-1).reshape(-1,1)),np.array((np.argmax(preds,axis=1)).reshape(-1,1)),average="macro",zero_division=0))
            precision_sublist = np.append(precision_sublist,precision_score(np.array(y.data.view(-1).reshape(-1,1)),np.array((np.argmax(preds,axis=1)).reshape(-1,1)),average="macro",zero_division=0))
            recall_sublist = np.append(recall_sublist,recall_score(np.array(y.data.view(-1).reshape(-1,1)),np.array((np.argmax(preds,axis=1)).reshape(-1,1)),average="macro",zero_division=0))

            optimizer.step()

            # Zero out any previously calculated gradients
            model.zero_grad()


            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Reset batch tracking variables
        batch_loss, batch_counts = 0, 0
        t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = np.mean(loss_sublist)
        tr_ep_loss.append(avg_train_loss)
        
        total_accuracy = np.mean(acc_sublist)
        tr_ep_acc.append(total_accuracy)

        total_f1 = np.mean(f1_sublist)
        tr_ep_f1.append(total_f1)
        
        total_precision = np.mean(precision_sublist)
        tr_ep_precision.append(total_precision)
        
        total_recall = np.mean(recall_sublist)
        tr_ep_recall.append(total_recall)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy, val_f1, val_precision, val_recall = final_evaluate(model, val_dataloader_final)
            
            if val_f1>best_val_f1:
                best_model_final1=copy.deepcopy(model)
                model_name_location = f"/kaggle/working/best_model_mafaulda.pt"
                torch.save(best_model_final1.state_dict(), model_name_location)
                best_val_f1 = val_f1
                print("This is best!")
                
            val_ep_loss.append(val_loss)
            val_ep_acc.append(val_accuracy)
            val_ep_f1.append(val_f1)
            val_ep_precision.append(val_precision)
            val_ep_recall.append(val_recall)

            time_elapsed = time.time() - t0_epoch

            # Print the header of the result table
            print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Accuracy':^12} | {'Train F1 score':^12} |{'Train Precision score':^12} |{'Train Recall score':^12} | {'Val Loss':^12} | {'Val Accuuracy':^12} | {'Val F1 score':^12}|{'Val Precision score':^12}|{'Val Recall score':^12}| {'Elapsed':^9}")
            print("-"*81)
            print(f"{e:^7} | {avg_train_loss:^12.6f} | {total_accuracy:^14.6} | {total_f1:^14.6} | {total_precision:^14.6} |{total_recall:^14.6} |{val_loss:^12.6f} | {val_accuracy:^14.6f} | {val_f1:^14.6f}|{val_precision:^14.6f}|{val_recall:^14.6f}| {time_elapsed:^9.2f}")
            print("-"*81)
        print("\n")

    torch.save(best_model_final1.state_dict(), '/kaggle/working/final_model_mafaulda_1.pt')
    
    return best_val_f1
    

In [ ]:
device = "cuda:0"
# device = "cpu"

In [ ]:
import math

In [ ]:
class KAN_model(nn.Module):

    def __init__(self):
        super(KAN_model, self).__init__()
        self.KAN1 = KAN([36, 100], grid_size=10, spline_order=1, grid_range=[-1, 4])
        self.projection = KAN([100, 6], grid_size=10, spline_order=1, grid_range=[-1, 1])

    def forward(self, x):
        x = x.view(x.shape[0], 1,-1)
        out = self.KAN1(x)
        out = self.projection(out)
        out_f = out.view(x.shape[0], out.size(1) * out.size(2))
        return out_f

In [ ]:
model_final = KAN_model()
model_final.to(device)
optimizer = torch.optim.AdamW(model_final.parameters(), lr=0.004, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.05, amsgrad=False)

In [ ]:
batch_size = 800

train_dataloader_final, val_dataloader_final = Create_Dataloader_final1(x_train_standardized, x_test_standardized, labels_train, labels_test, batch_size)

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model_final.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

In [ ]:
 #final training
epoch = 200
final_train(model_final, optimizer, epoch, train_dataloader_final, val_dataloader_final, evaluation=True)

In [ ]:
best_model_path = "/kaggle/working/best_model_mafaulda.pt"

best_model = KAN_model()
best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Step 1: Run inference to get predictions
y_true = []
y_pred = []

best_model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model.to(device)
best_model.float()

with torch.no_grad():
    for data, target in val_dataloader_final:  # assuming (data, label)
        data, target = data.to(device), target.to(device)
        data = data.float()
        outputs = best_model(data)
        preds = torch.argmax(outputs, dim=1)
        y_true.extend(target.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Step 2: Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Optional: class names (if known, replace with actual class names)
class_names = ['DownStairs', 'Jogging', 'Sitting', 'Standing', 'UpStairs', 'Walking']

# Step 3: Plot with seaborn
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.tight_layout()
# plt.show()
plt.savefig("conf_matrix.eps")
